In [2]:
import pandas as pd
import re

In [3]:
data=pd.read_csv('data_prepared_time.csv')

In [4]:
data.head()

,Unnamed: 0,Unnamed: 0.1,thread_id,thread_dayofweek,thread_hour,thread_timestamp,text,time_deltas,reply_hour,reply_dayofweek,channel,reply_text,reply_timestamp
0,0,0,12781837000,3,9,2017-01-19 09:06:10,<@U1G303UTW> get laid with Bayes,193,9,3,_call_4_collaboration,<@U30Q72KLJ> он же вроде умер,2017-01-19 09:09:23
1,1,1,12781879600,3,9,2017-01-19 09:13:16,"Чатег, я вас разочарую - сообщение в дейтинге ...",57,9,3,_call_4_collaboration,<@U2TP5JELS> а как ты проверял?,2017-01-19 09:14:13
2,2,2,12782089400,3,9,2017-01-19 09:48:14,"Из `{&lt;женское_имя&gt;; уменьшительное, ласк...",49,9,3,_call_4_collaboration,"`{&lt;женское_имя&gt;; уменьшительное, ласкате...",2017-01-19 09:49:03
3,3,3,12844819600,3,16,2017-01-26 16:03:16,Гарри Селдон в восхищении от вашего проекта.,408,16,3,_call_4_collaboration,notabene: ну все-таки надо добавить был бы,2017-01-26 16:10:04
4,4,4,12844915400,3,16,2017-01-26 16:19:14,"а можете рассказать, в чем заключается вообще ...",30652,0,4,_call_4_collaboration,"katya: позже напишу, как будет время:)",2017-01-27 00:50:06


In [5]:
#просто перевожу назву каналу в числа, щоб можно було його як фічу використовувати 
channel_names = data['channel'].unique()
channel_names_dictionary = {}

for i, channel in enumerate(channel_names):
    channel_names_dictionary[channel] = i
    
data['channel'] = pd.Series([channel_names_dictionary[channel] for channel in data['channel']], dtype=int)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer



In [7]:
data.head()

,Unnamed: 0,Unnamed: 0.1,thread_id,thread_dayofweek,thread_hour,thread_timestamp,text,time_deltas,reply_hour,reply_dayofweek,channel,reply_text,reply_timestamp
0,0,0,12781837000,3,9,2017-01-19 09:06:10,<@U1G303UTW> get laid with Bayes,193,9,3,0,<@U30Q72KLJ> он же вроде умер,2017-01-19 09:09:23
1,1,1,12781879600,3,9,2017-01-19 09:13:16,"Чатег, я вас разочарую - сообщение в дейтинге ...",57,9,3,0,<@U2TP5JELS> а как ты проверял?,2017-01-19 09:14:13
2,2,2,12782089400,3,9,2017-01-19 09:48:14,"Из `{&lt;женское_имя&gt;; уменьшительное, ласк...",49,9,3,0,"`{&lt;женское_имя&gt;; уменьшительное, ласкате...",2017-01-19 09:49:03
3,3,3,12844819600,3,16,2017-01-26 16:03:16,Гарри Селдон в восхищении от вашего проекта.,408,16,3,0,notabene: ну все-таки надо добавить был бы,2017-01-26 16:10:04
4,4,4,12844915400,3,16,2017-01-26 16:19:14,"а можете рассказать, в чем заключается вообще ...",30652,0,4,0,"katya: позже напишу, как будет время:)",2017-01-27 00:50:06


In [6]:
# можна і прибрати цей макс_фічерс але тоді буде дуже велика матриця (50000 десь)
cv = CountVectorizer(
                    analyzer='word', # token = word
                    ngram_range=(1,1), # only unigrams are used, (1,2) - unigrams/bigrams, ..., etc.
                    #stop_words=['my', 'stop', 'word', 'list'], # or stop_words='english'
                    vocabulary=None, # or vocabulary=your_own_dictionary
                    max_df=1.0, # don't filter words by their frequency
                    max_features=1000 # only top-1000 words will be used as columns
                    )

In [7]:
# формуємо матрицю, яка складається з векторів - які відповідають тексту повідомлень
# координати вектора позначають слова
# (розмірність вектора це кількість унікальних слів в нашому датасеті).
transformed_texts_cv = cv.fit_transform(data['text'].values.astype('U'))

In [8]:
print('Obtained feature matrix X:')
A_cv=transformed_texts_cv.todense()
print(transformed_texts_cv.todense(), '\n')

Obtained feature matrix X:
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]] 



In [9]:
A_cv.shape

(12561, 1000)

In [10]:
import numpy as np
A_cv_1=np.asarray(A_cv)

In [12]:
(A_cv_1).shape[1]

1000

In [13]:
# ще додатоково обираю 3 фічі - день тиждня і годину коли повідомлення було написано і канал 
data1=data[['thread_dayofweek','thread_hour','channel']]

In [14]:
A_1 = data1.as_matrix()

In [15]:
print(A_1.shape)
print(A_cv_1.shape)

(12561, 3)
(12561, 1000)


In [16]:
#остаточно отримую свою features matrix 
A_feat=np.concatenate((A_1,A_cv_1),axis=1)

In [17]:
A_feat.shape

(12561, 1003)

In [18]:
# time_deltas це час(в секундах) коли відповіли на повідомлення 
data2=data['time_deltas']
y=data2.as_matrix()
y.shape

(12561,)

In [19]:
print(y)

[  193    57    49 ..., 61688 61149   510]


In [20]:
A_feat.shape

(12561, 1003)

In [26]:
# власне треную свою модель, використовуючи lasso регресію, 
# можна спробувати ділити датасет не рандомом, а ,наприклад, тест брати все після 1 травня чи якось так 
from sklearn import linear_model
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(A_feat, y, test_size=0.3, random_state=42)

#16-18

reg_1 = linear_model.Lasso(alpha = 0.1,max_iter=1000)
reg_1.fit(X_train, y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [32]:
import numpy as np
#np.sqrt(mse(y_test, reg_1.predict(X_test)))/60

251.21297201988153

In [33]:
# mae я вже дивлюся в хвилинах - шоб не так печально было)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test/60, (reg_1.predict(X_test))/60)

149.70963097455001

In [28]:
r2_score(y_test, reg_1.predict(X_test))

-0.21257323028281849